# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [49]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Set Up Environment

In [83]:
network = "local"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_testnet_20201016125222.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [75]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# with open(contracts_file, 'rb') as f:
#     contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [84]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [56]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [42]:
icon_service.get_balance("hxe7af5fcfd8dfc67530a01a0e403882687528dfcb")

800459514056929999999990000

### Importing wallet from keystore file

In [87]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /api/v3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb022479fd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [57]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [23]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token']
# for directory in {"score"}:
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as contracts_20201106101231.pkl


{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token.zip'},
 'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'}}

In [ ]:
# Deploying contracts

In [86]:
# params = {'_initialSupply':500000000, '_decimals': 18}
params = {}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('addressProvider'))\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /api/debug/v3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb0229b3a60>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [79]:
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
lendingPoolDataProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip', 'SCORE': 'cxbb0dce98acd5ad61019ef18271064e6900f0ab65'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 4 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending tran

KeyboardInterrupt: 

# Updating a contract

In [36]:
contract = 'lendingPool'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xa77c8cd5457c93c267a472e685a67825802efaef9e4902dd77be6ea7b0a29c5b',
 'blockHeight': 96428,
 'blockHash': '0xd805acfe7df5115318a02ea7e339dd6fcb55ff378577fc921aeaebc1d622876b',
 'txIndex': 1,
 'to': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'scoreAddress': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'stepUsed': 1635708000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1635708000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Configuring SCOREs's parameters

In [66]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 25*10**14}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':5*10**17}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":"0","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}}]

In [67]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

ds...
{'txHash': '0x01b72ec2ae7198ac61773b8d8cbdf9e07f75107fa77dc1f46571c055fd0dc805', 'blockHeight': 313089, 'blockHash': '0x478ec481789c554a02be04d7458caf5f1d3ad6ed9e4d134ee8328daf9adba7f8', 'txIndex': 1, 'to': 'cxbb0dce98acd5ad61019ef18271064e6900f0ab65', 'stepUsed': 147680, 'stepPrice': 10000000000, 'cumulativeStepUsed': 147680, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding new reserve in LendingPoolCore



In [68]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60000000000000000000","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxec46d0c467362c5d8271f397e50be3afb52679ba', 'stepLimit': '0x59920', 'timestamp': '0x5b36e96135c0a', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'addReserveData', 'params': {'_reserve': {'reserveAddress': 'cx2117116d4137b577c9609f0b58d391561ed40870', 'oTokenAddress': 'cxaffc4824001c20a65677fd2e33ba8f0701951121', 'totalBorrows': '0', 'lastUpdateTimestamp': '0', 'liquidityRate': '0', 'borrowRate': '0', 'liquidityCumulativeIndex': '1000000000000000000', 'borrowCumulativeIndex': '1000000000000000000', 'baseLTVasCollateral': '60000000000000000000', 'liquidationThreshold': '65', 'liquidationBonus': '10', 'decimals': '18', 'borrowingEnabled': '1', 'usageAsCollateralEnabled': '1', 'isFreezed': '0', 'isActive': '1'}}}, 'signature': 'J73kpSu8mNpF9XGW91B9eRVj/wJiDMPxCxUzDoJ4HCEEXuSgJKtXHNybO32eeoNCsoo/Ot2XrLYSbCRZzHz+8wE='}
Raised JSONRPCException while returning the custom response. Error me

{'txHash': '0xa7eacfa5690ae049cc6d4f9bc779cbfdf6c91e0a029620a567a1eff4377441f4',
 'blockHeight': 313117,
 'blockHash': '0x973f77fc778b142d1f46f7da45f7bb224b2bfdaa796ebe310724e1fc2da13255',
 'txIndex': 1,
 'to': 'cxec46d0c467362c5d8271f397e50be3afb52679ba',
 'stepUsed': 266880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 266880,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Depositing USDb

In [ ]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Borrowing USDb

In [40]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":1*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x118e38', 'timestamp': '0x5b35a05b0f8a4', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', '_amount': '0xde0b6b3a7640000'}}, 'signature': 'qUPcgNJ6GAnY5QMfYNtnvlPGztPNQiK2vpx0XUNcZiRi7K8sTPOQFkv5uwCjtJxNZ7rVB7oW8P2Kayd6Kp+ChwA='}


{'txHash': '0x37d376bbec9f44cea3a4d2256d921ae1907daf28832b43e4afe8127c368f9a9c',
 'blockHeight': 269010,
 'blockHash': '0x600fb1885bc071cb1426c485e0dc85f0841c907c1a54abd1aa8855cbe7a3c8eb',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 1050520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1050520,
 'eventLogs': [{'scoreAddress': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
    '0xde3de7f147',
    '0x13fec39f610b9'],
   'data': ['0xde0b6b3a7f89cc9', '0xde0b6b4fe72f867']},
  {'scoreAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0xde0b6b3a7640000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

### Getting reserve data

In [73]:
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
pprint(response)

{'availableLiquidity': '0x0',
 'baseLTVasCollateral': '0x340aad21b3b700000',
 'borrowCumulativeIndex': '0xde0b6b3a7640000',
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': '0x12',
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x0',
 'liquidationBonus': '0xa',
 'liquidationThreshold': '0x41',
 'liquidityCumulativeIndex': '0xde0b6b3a7640000',
 'liquidityRate': '0x0',
 'oTokenAddress': 'cxaffc4824001c20a65677fd2e33ba8f0701951121',
 'reserveAddress': 'cx2117116d4137b577c9609f0b58d391561ed40870',
 'totalBorrows': '0x0',
 'totalLiquidity': '0x0',
 'usageAsCollateralEnabled': '0x1'}


### Getting user account data

In [29]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER)\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 2.5838e-14,
 'currentLtv': 60.0,
 'healthFactor': 203882.82366782034,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.5000003816736971,
 'totalCollateralBalanceUSD': 397.5000009264478,
 'totalFeesUSD': 0.00375,
 'totalLiquidityBalanceUSD': 397.5000009264478}


### Getting user reserve data

In [28]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER)\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.000440251572327044,
 'currentBorrowBalance': 1.0000007629006646,
 'currentBorrowBalanceUSD': 0.5000003814503323,
 'currentOTokenBalance': 795.0000018516895,
 'currentOTokenBalanceUSD': 397.50000092584474,
 'lastUpdateTimestamp': 1604574108126959,
 'liquidityRate': 1.495194019224e-06,
 'originationFee': 0.0075,
 'principalBorrowBalance': 1.0,
 'principalBorrowBalanceUSD': 0.5,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 1.0000000057555662}


### Get reserve configuraitions


In [25]:
params = {"_reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42"}
# params={"_base":"USDb","_quote":"USD"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_CORE)\
    .method("getReserveConfiguration")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 60000000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 65,
 'usageAsCollateralEnabled': 1}


In [17]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .method("principalBalanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

795000001589272371715


In [18]:

params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(int(response,0))

795000001744001199280


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511